# BEFORE YOU START, CHANGE THE RUNTIME TO REQUEST A T4 GPU

In [1]:
!git clone https://github.com/BrendanCreates/Finetuning-Mistral-7B-Legal.git

Cloning into 'Finetuning-Mistral-7B-Legal'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 95 (delta 51), reused 14 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (95/95), 1.96 MiB | 5.98 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [2]:
# Installs
# You will need at a minimum the following packages. Feel free to install
# additional ones as needed
!pip install google-generativeai
!pip install datasets
!pip install -U bitsandbytes
!pip install transformers
!pip install -U peft
!pip install -U "huggingface_hub[cli]"
!pip install -U trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.0/516.0 kB 27.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.1 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.1.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 51.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface_hub 1.1.5
    Uninstalling huggingface_hub-1.1.5:
      Successfully uninstalled huggingface_hub-1.1.5


In [3]:
import google.generativeai as genai
from datasets import Dataset, DatasetDict
import pandas as pd
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, \
    BitsAndBytesConfig, TrainingArguments, pipeline, logging
import torch
from trl import SFTTrainer

In [18]:
# In this block, I include code for parsing the Intro chapter from a text file.
# I ran the following two lines on a linux terminal. You can find the equi-
# valent for your OS or find an online tool for converting PDF into text.
# >>> pdftotext -nopgbrk MIT_6390_chapter_Introduction.pdf
# >>> sed -r ':a /[a-zA-Z,\ ]$/N;s/(.)\n/\1 /;ta' \
#        MIT_6390_chapter_Introduction.txt > \
#        MIT_6390_chapter_Introduction_reformat.txt
#
# Once the PDF was converted to a text file, I manually looked through it to:
# - Remove ninformative lines (e.g., "Last updated: ...", "MIT 6390", ...)
# - Remove comments, which come somewhat poorly organized
# - Remove double line breaks
# - Fix up equations a bit so that they made sense in text format
#
# I did all this in a simple text editor (I used Sublime Text). Then I simply
# uploaded the file to Colab and ran the following code to split the text into
# informative paragraphs. This required a bit of iterating back and forth to
# make sure that no paragraph was "trailing" from the previous one.

import glob

paragraphs = []

for file in glob.glob("/content/Finetuning-Mistral-7B-Legal/data/*.txt"):
  with open(file) as f:
    lines = f.readlines()

  min_chars = 200
  par = lines[0]
  for ln in lines[1:]:
    if ln[0] == "•" or ln[0].isdigit() and ln[1:3] == ". ":
      # Part of a list, combine with previous items
      par += ln
    else:
      paragraphs.append(par.strip())  # Remove trailing whitespace and store
      par = ln               # Start new paragraph

  paragraphs.append(par)

for line in paragraphs:
  print(line)
  print('---')
print(len(paragraphs))

# I have uploaded the reformatted text file I used to generate data for the
# Intro chapter. You should follow a similar process to create data from any
# source you wish to use. Note: the better you clean up your data, the more
# useful your final model will be.

{"input": "Question: In a fixed-price construction contract where the owner provides detailed plans and requires the contractor to follow them, what is the usual effect if those plans are defective?\nChoices:\nA. The contractor must fix all consequences of the defect at its own cost because it agreed to a fixed price.\nB. The owner is treated as having impliedly warranted the adequacy of the plans, so the owner may bear responsibility for defects in them.\nC. The contractor and owner automatically split the extra costs 50–50 unless the contract says otherwise.\nD. The contractor is excused from finishing the job but cannot recover any money.", "output": "Correct: B\nExplanation: In many U.S. jurisdictions, when an owner furnishes specific plans and specifications and requires the contractor to follow them, the law implies a warranty that those plans are reasonably adequate for their intended purpose. If the contractor builds as required and a failure is caused solely by design defects,

In [22]:
qa_pairs_val = [
  "<s>[INS]@ESE577.In a fixed-price construction contract where the owner provides detailed plans and requires the contractor to follow them, what is the usual effect if those plans are defective?\nChoices:\nA. The contractor must fix all consequences of the defect at its own cost because it agreed to a fixed price.\nB. The owner is treated as having impliedly warranted the adequacy of the plans, so the owner may bear responsibility for defects in them.\nC. The contractor and owner automatically split the extra costs 50–50 unless the contract says otherwise.\nD. The contractor is excused from finishing the job but cannot recover any money.[/INS]Correct: B\nExplanation: In many U.S. jurisdictions, when an owner furnishes specific plans and specifications and requires the contractor to follow them, the law implies a warranty that those plans are reasonably adequate for their intended purpose. If the contractor builds as required and a failure is caused solely by design defects, courts often place responsibility on the owner, not the contractor, despite the fixed price. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What is the main purpose of a “differing site conditions” clause in a construction contract?\nChoices:\nA. To guarantee that the contractor will never face unexpected subsurface conditions.\nB. To shift all risk of unknown site conditions to the owner in every situation.\nC. To provide a mechanism for adjusting time or price when actual conditions materially differ from those indicated or normally expected.\nD. To allow the owner to terminate the contractor at any time without paying for completed work.[/INS]Correct: C\nExplanation: A differing site conditions clause does not eliminate surprises, but it sets out a process for adjusting time and payment when conditions are materially different from those described or ordinarily found. This encourages realistic bids and reduces the need for contractors to build massive contingency amounts into their prices. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Which of the following best describes a Type II differing site condition?\nChoices:\nA. A condition that is exactly as described in the contract documents.\nB. An unknown, unusual physical condition that differs materially from what is normally encountered in similar work.\nC. Any subsurface condition that increases the contractor’s costs, even if common in the area.\nD. A condition that the contractor could easily have discovered during a pre-bid site visit.[/INS]Correct: B\nExplanation: Type II differing site conditions involve unusual physical conditions that are not specifically indicated in the contract but differ materially from those ordinarily found in similar projects. The contractor must show the condition was unknown, could not reasonably be anticipated, and is different from the norm. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.A tunnel contractor signed a change order agreeing to use a different construction method and stating it “assumed responsibility” for that method, but the documents did not mention the existing differing site conditions clause. Later, the contractor encountered a toxic substance no one expected. How is a court most likely to treat the contractor’s differing site conditions rights?\nChoices:\nA. The rights are waived whenever the contractor signs any change order.\nB. The rights survive unless the change order clearly shows an intent to waive protection for unforeseen hazards.\nC. The rights automatically shift to the architect.\nD. The rights are converted into a tort claim for fraud.[/INS]Correct: B\nExplanation: Courts typically read change orders together with the original contract. If the change order addresses methods but does not clearly waive the differing site conditions clause, many judges will conclude that the protection for truly unforeseen, extraordinary hazards remains in place. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.A subcontract states that the subcontractor “accepts the site as-is and assumes all risk of subsurface conditions, whether known or unknown,” and also says there will be no extra compensation for changed conditions. What is the most likely legal effect of this language?\nChoices:\nA. The subcontractor can still claim extra payment if it subjectively did not expect the condition.\nB. The owner must pay for any extra costs because construction always involves risk.\nC. The subcontractor has contractually taken on the risk of subsurface surprises and will usually struggle to get extra money for them.\nD. The language is automatically void as against public policy.[/INS]Correct: C\nExplanation: Clear “as-is” and assumption-of-risk language, especially when coupled with a no-extra-compensation clause, typically means the subcontractor has agreed to bear the financial impact of unexpected subsurface conditions. Courts often enforce this allocation in commercial settings, leaving the subcontractor to absorb cost overruns unless another theory like fraud applies. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What is the usual effect of a merger or integration clause in a construction contract?\nChoices:\nA. It guarantees that all oral promises are enforceable.\nB. It states that the written contract is the full and final agreement, limiting reliance on prior statements or documents.\nC. It voids the contract if any changes are later made in writing.\nD. It automatically extends all warranties to future owners.[/INS]Correct: B\nExplanation: A merger or integration clause tells the court that the parties intend the written contract to be the complete agreement, superseding prior negotiations and representations. This makes it harder to base claims on pre-bid conversations or documents that were not incorporated into the final contract. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.A subcontractor signs a clause stating that in entering the subcontract it “relied solely on its own investigation and not on any opinions or representations of the contractor” about soil conditions. Which statement is most accurate?\nChoices:\nA. The subcontractor can still easily argue it reasonably relied on the contractor’s estimates.\nB. The clause will often be used to undermine later claims of reliance on the contractor’s soil information.\nC. The clause has no legal effect because it is boilerplate.\nD. The clause automatically requires the contractor to pay for any extra costs.[/INS]Correct: B\nExplanation: No-reliance language is specifically designed to undercut later claims that the subcontractor reasonably relied on pre-contract estimates or statements. Courts often give this language significant weight when deciding whether the subcontractor can pursue misrepresentation or differing site conditions theories based on the contractor’s earlier information. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Under modern owner–architect agreements, which of the following best describes the architect’s role regarding job-site safety?\nChoices:\nA. The architect has day-to-day control over all safety decisions and is responsible for worker injuries.\nB. The architect is solely responsible for installing safety equipment.\nC. The architect makes periodic site visits and “endeavors to guard” the owner against defects, but the contractor is responsible for means, methods, and safety programs.\nD. The architect is not allowed to enter the site during construction.[/INS]Correct: C\nExplanation: Modern contracts typically state that the architect will visit the site at intervals to observe progress and general quality but will not control construction means, methods, or safety. Responsibility for safety programs and day-to-day precautions is assigned to the contractor, though an architect can still face liability if it actually takes control and acts negligently. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.If an architect with broad contractual power to supervise the work sees an obviously dangerous method being used and does nothing, what is a possible legal consequence?\nChoices:\nA. The architect is automatically criminally liable.\nB. The architect may be found to have breached a duty of care and face civil liability for resulting injuries.\nC. The contractor is completely absolved of responsibility.\nD. The workers lose their right to workers’ compensation.[/INS]Correct: B\nExplanation: When an architect undertakes active supervision and has the authority to stop unsafe work, some courts have held that failing to act in the face of an obvious danger can breach the architect’s duty of care. That can lead to civil liability for injuries reasonably connected to the unsafe practice, alongside the contractor’s responsibilities. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Which pair of elements most accurately reflects what courts often require to pierce the corporate veil?\nChoices:\nA. Owner domination and a history of making profits.\nB. Owner domination and use of that domination to commit a wrong that injures the claimant.\nC. Owner domination and a very small number of employees.\nD. Owner domination and a long-term lease of office space.[/INS]Correct: B\nExplanation: Veil-piercing analysis usually focuses on whether the owner exercised domination over the company and whether that control was used to commit fraud, evade obligations, or otherwise work an injustice that harmed the claimant. Domination plus benign conduct is generally not enough. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Which of the following facts is most likely to support piercing the corporate veil of a small construction company?\nChoices:\nA. The company has a website and business cards.\nB. The owner occasionally signs letters on company letterhead.\nC. The owner routinely uses the company bank account to pay personal household bills.\nD. The company’s office is in a shared coworking space.[/INS]Correct: C\nExplanation: Commingling personal and corporate funds—such as using the company account to pay personal bills—is one of the classic indicators courts look at when deciding whether the corporate form has been abused. While not sufficient by itself, it strongly suggests the owner is not respecting the company as a separate entity. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.A building owner sues a tile manufacturer in strict products liability seeking only the cost to replace defective tiles that delaminate, with no injury or damage to other property. How do many courts treat such a claim?\nChoices:\nA. As a typical strict liability case, because any product defect is a tort matter.\nB. As primarily a contract/warranty dispute, limiting or barring tort recovery for pure economic loss.\nC. As a criminal fraud case.\nD. As a workers’ compensation matter.[/INS]Correct: B\nExplanation: When the harm is purely economic—such as the cost to repair or replace the defective tiles—many courts apply the economic loss rule and treat the dispute as one for contract and warranty, not strict tort liability. Tort remedies are usually reserved for personal injury or damage to other property. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.In which situation is strict products liability most likely to be available in a construction materials case?\nChoices:\nA. Defective roofing shingles require replacement, but nothing else is damaged.\nB. Faulty windows slightly reduce energy efficiency but cause no other harm.\nC. Defective tiles cause a tenant to fall and suffer a serious personal injury.\nD. Carpet color fades faster than expected but still functions.[/INS]Correct: C\nExplanation: Strict products liability is aimed at protecting people and property from unreasonable safety risks. When a defective product causes personal injury, tort remedies are much more likely to be available than when the only harm is dissatisfaction with performance or cost to replace the product. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What is the best description of “indemnification” in the context of a subcontractor and manufacturer?\nChoices:\nA. A promise by the owner never to sue anyone.\nB. A mechanism by which one party agrees to reimburse another for losses arising from certain claims, such as defects in supplied products.\nC. A requirement that the subcontractor work for free.\nD. A criminal penalty for safety violations.[/INS]Correct: B\nExplanation: Indemnification is a contractual or common-law right that allows a party who pays damages (for example, a subcontractor liable to an owner) to shift those losses to another party (such as a manufacturer) who is ultimately responsible. It is about reimbursement for specified types of claims, not about immunity from all lawsuits. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Why are payment bonds or guaranties especially important when private construction occurs on publicly owned land?\nChoices:\nA. Because public land is immune from environmental regulation.\nB. Because contractors usually cannot file mechanics’ liens against public property.\nC. Because public projects never involve subcontractors.\nD. Because courts refuse to hear payment disputes involving public landlords.[/INS]Correct: B\nExplanation: Mechanics’ liens generally cannot be placed on public property, so contractors and subcontractors might otherwise have little leverage if not paid. Payment bonds or guaranties offer an alternative source of recovery, ensuring there is security behind the owner’s payment obligations. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.A contractor signs a contract with a thinly capitalized special-purpose entity that owns nothing but the project and has no guaranty. What is a key risk?\nChoices:\nA. The contractor will automatically be paid early.\nB. The contractor may win a judgment but have no financially viable entity from which to collect.\nC. The contractor gains a right to pierce the veil without proof.\nD. The contractor cannot be sued by anyone else.[/INS]Correct: B\nExplanation: If the project company runs out of money or goes bankrupt, a contractor may be left with an uncollectible judgment. Without additional protection like a bond or guaranty, the contractor bears significant credit risk that the special-purpose owner may not be able to pay what is owed. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Absent special contract clauses, who usually bears the risk that soil at a building site is more difficult than expected, such as being harder to excavate?\nChoices:\nA. The architect.\nB. The contractor, who agreed to do the work for a fixed sum.\nC. The building inspector.\nD. The local government.[/INS]Correct: B\nExplanation: Under general rules, a contractor who agrees to perform work for a fixed price is not excused or entitled to more money solely because it encounters unforeseen difficulties like harder soil, unless the contract shifts that risk or the owner’s plans are defective. The contractor is normally expected to account for such possibilities when pricing the job. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How do differing site conditions clauses help produce more realistic bids?\nChoices:\nA. By forcing all contractors to submit identical prices.\nB. By allowing contractors to bid based on indicated conditions, knowing that truly unexpected conditions can be addressed by later adjustments.\nC. By guaranteeing that no changes will ever be made to the work.\nD. By eliminating the need for contractors to investigate the site.[/INS]Correct: B\nExplanation: When contractors know they can seek adjustments if they encounter material differences from the conditions shown or ordinarily expected, they do not need to include huge contingency amounts in their bids. This leads to more competitive and realistic pricing for owners and fairer risk allocation for contractors. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Even with an integration and no-reliance clause, in what situation might a contractor still be able to sue an owner for misrepresentation?\nChoices:\nA. If the contractor simply regrets signing the contract.\nB. If the owner intentionally concealed a major hazard the contractor could not reasonably discover.\nC. If the contractor failed to read the contract.\nD. If market prices for labor go up.[/INS]Correct: B\nExplanation: Many courts allow fraud or intentional misrepresentation claims despite integration and no-reliance clauses when there is strong evidence that one party deliberately hid or lied about a material fact. The contractor still must prove the elements of fraud, but intentional concealment can sometimes overcome otherwise enforceable disclaimers. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.A subcontract clearly states that the subcontractor assumes all risk for differing subsurface conditions and will receive no additional compensation if actual conditions differ from any estimates. Later, the subcontractor encounters more contaminated soil than expected. Which outcome is most likely?\nChoices:\nA. The subcontractor will easily obtain a large change order.\nB. The subcontractor will usually be held to its bargain and may not recover extra money for the additional contaminated soil.\nC. The owner must automatically pay all extra costs.\nD. The clause will be ignored because contaminated soil is always unexpected.[/INS]Correct: B\nExplanation: When commercial parties clearly allocate subsurface risk to the subcontractor and disclaim extra compensation for differing conditions, courts often enforce that arrangement. The subcontractor has effectively priced and accepted that risk, making later claims for more money difficult. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.When a contractor believes a specified design creates an immediate safety hazard, what is generally the safest legal and practical step?\nChoices:\nA. Continue work as usual and raise the concern after completion.\nB. Immediately stop work, document the concern, and notify the owner or architect in writing.\nC. Secretly change the design without telling anyone.\nD. Fire all workers on site.[/INS]Correct: B\nExplanation: Stopping work in the affected area and promptly notifying the owner or architect in writing helps protect workers’ safety and preserves the contractor’s legal position. Proceeding “at its peril” despite known hazards can expose the contractor to liability and weaken its argument that the design was unacceptably dangerous. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Which statement best describes a Type I differing site condition?\nChoices:\nA. A condition worse than the contractor subjectively expected, but consistent with the contract documents.\nB. A condition materially different from what the contract affirmatively represented or indicated.\nC. Any soil contamination regardless of contract language.\nD. Any condition that causes delay, even if it was obvious.[/INS]Correct: B\nExplanation: Type I differing site conditions involve a mismatch between the conditions indicated in the contract documents and the conditions actually encountered. The key is that the contract made an affirmative representation that turns out to be materially inaccurate. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How is indemnity different from contribution between defendants in a construction defect case?\nChoices:\nA. Indemnity is criminal, contribution is civil.\nB. Indemnity seeks full reimbursement from another party, while contribution typically seeks to share losses among parties based on fault.\nC. Indemnity only applies to architects, contribution only to owners.\nD. They are legally identical terms.[/INS]Correct: B\nExplanation: Indemnity usually involves one party shifting the entire burden of a loss to another, while contribution involves multiple parties sharing the loss in proportion to their respective fault. Both concepts can appear in construction disputes, but they operate differently in allocating costs among responsible actors. This is general information about U.S. new. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Which contract feature most increases the likelihood that an architect will be found to have a duty regarding job-site safety?\nChoices:\nA. Contract language expressly disclaiming responsibility for safety and means and methods.\nB. Contract language granting the architect broad power to supervise the work and stop unsafe operations.\nC. Contract language limiting the architect’s role to reviewing shop drawings.\nD. Contract language requiring the architect to send monthly invoices.[/INS]Correct: B\nExplanation: When an architect’s contract grants broad supervisory power and authority to stop work for safety reasons, courts may be more inclined to find a duty to act reasonably when dangerous conditions are apparent. Disclaimers and limited roles generally reduce, but do not always eliminate, such duties. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.A contract states that the contractor must repair damage “except such as may be directly due to errors in the contract documents or caused by the owner’s agents.” If damage occurs solely because the structural drawings were wrong, what is the most likely result?\nChoices:\nA. The contractor must still repair the damage at its own cost.\nB. The contractor is relieved from paying for that damage because it stems from errors in the owner’s documents.\nC. The contract is automatically void.\nD. The architect becomes the contractor’s employee.[/INS]Correct: B\nExplanation: This clause carves out an exception for damage caused directly by errors in the owner’s documents or agents’ acts, meaning the contractor is not contractually responsible for such damage. The owner and its design professionals are more likely to bear that cost. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What is one of the best ways for a building owner to protect itself against economic loss from defective materials like floor tiles?\nChoices:\nA. Rely solely on strict products liability law.\nB. Negotiate clear express warranties and allocate responsibility for product failures in its contracts.\nC. Avoid putting anything in writing.\nD. Require contractors to work without profit.[/INS]Correct: B\nExplanation: Because tort remedies for pure economic loss are often limited, owners are best protected by negotiating warranties and risk-allocation clauses that spell out who must pay for repairs or replacements if materials fail. Clear contractual protections are more reliable than hoping tort law will fill gaps later. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.When a contractor unexpectedly encounters hazardous material on site, which of the following actions best protects both safety and contractual rights?\nChoices:\nA. Ignoring the hazard to avoid delays.\nB. Continuing work but making a mental note of the problem.\nC. Stopping work in the affected area, notifying the owner in writing, and following contractual claim procedures.\nD. Immediately abandoning the entire project without notice.[/INS]Correct: C\nExplanation: Stopping work where necessary for safety, documenting the condition, and giving prompt written notice aligns with both safety responsibilities and the notice requirements of most differing site conditions clauses. This approach helps preserve the contractor’s right to seek time or money adjustments while reducing risk to workers. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Which clause is typically most dangerous for a small subcontractor from a risk standpoint?\nChoices:\nA. A clause requiring weekly progress meetings.\nB. A clause stating that the subcontractor will not receive additional compensation for changed site conditions and assumes all subsurface risks.\nC. A clause requiring the subcontractor to provide its own tools.\nD. A clause specifying that invoices are due within 30 days.[/INS]Correct: B\nExplanation: A clause that both assumes all subsurface risks and bars extra compensation for changed conditions can expose a subcontractor to enormous, unbounded costs if unforeseen problems arise. Such language is a major shift from standard risk allocation and should be carefully evaluated. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.In which scenario is a court least likely to pierce the corporate veil of a project company?\nChoices:\nA. The owner siphons funds out of the company while leaving suppliers unpaid.\nB. The company is used to commit fraud and then dissolved.\nC. The company has separate books, adequate capitalization, and observes corporate formalities even though the project loses money.\nD. The owner uses the company account to pay personal mortgage payments.[/INS]Correct: C\nExplanation: When a company respects formalities, is adequately capitalized, and is operated as a genuine separate entity, courts are generally reluctant to pierce the corporate veil, even if the project ultimately fails. Veil piercing is reserved for situations where the corporate form is abused to perpetrate a wrong. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.A defective building component fails and causes both property damage to surrounding structures and serious injuries to occupants. How are courts most likely to treat the owner’s claims against the manufacturer?\nChoices:\nA. As barred by the economic loss rule because only the product itself matters.\nB. As potentially valid tort claims because of personal injury and damage to other property.\nC. As limited to workers’ compensation.\nD. As automatically dismissed because the product was installed in real estate.[/INS]Correct: B\nExplanation: When a defective product causes personal injuries or damage to property beyond the product itself, courts are far more willing to apply tort doctrines like strict products liability or negligence. The economic loss rule is usually aimed at pure financial loss without such broader harm. This is general information about U.S.Log. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>"
]

In [24]:
"""
sys_msg_val = (

'''

 For this json line, strictly and ONLY reformat the line as so where you would replace <question> with what is inside of the quotes after {"input": in the json line, and replace <answer> with what is inside the quotes after "output": to convert the line as so:
 <s>[INS]@ESE577.<question>[/INS]<answer></s>

'''

)
print(sys_msg_val)
print()
model_val = genai.GenerativeModel('gemini-2.5-flash', system_instruction=sys_msg_val)
qa_pairs_val = []
for par in paragraphs[:30]:
  qa_pairs_val.append(model_val.generate_content(par, generation_config=cfg).text)
  print(qa_pairs_val[-1])
"""

'\nsys_msg_val = (\n\n\'\'\'\n\n For this json line, strictly and ONLY reformat the line as so where you would replace <question> with what is inside of the quotes after {"input": in the json line, and replace <answer> with what is inside the quotes after "output": to convert the line as so:\n <s>[INS]@ESE577.<question>[/INS]<answer></s>\n\n\'\'\'\n\n)\nprint(sys_msg_val)\nprint()\nmodel_val = genai.GenerativeModel(\'gemini-2.5-flash\', system_instruction=sys_msg_val)\nqa_pairs_val = []\nfor par in paragraphs[:30]:\n  qa_pairs_val.append(model_val.generate_content(par, generation_config=cfg).text)\n  print(qa_pairs_val[-1])\n'

In [25]:
qa_pairs_train = [
  "<s>[INS]@ESE577.What does it mean to “pierce the corporate veil” in the context of a small construction or service company?[/INS]Piercing the corporate veil means a court is willing to ignore the usual protection that an LLC or corporation gives its owners and treat the owners’ personal assets as available to satisfy business debts or judgments. Normally, forming an LLC or corporation shields the owner from personal liability, but if the entity is misused—such as being treated as the owner’s personal piggy bank or used to commit a wrong—a court may decide that equity requires holding the individual owner responsible. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What two main elements generally must be shown for a court to pierce the corporate veil and reach an owner’s personal assets?[/INS]Courts usually require a showing of (1) complete or near-complete domination of the company by its owner with respect to the particular transaction, and (2) that this domination was used to commit a fraud or other wrong that caused the claimant’s injury. In other words, it is not enough that the owner is in charge; the owner must have abused the corporate form to cause harm. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.If a homeowner wants to hold the individual owner of an LLC construction company personally liable for poor work, what do they generally need to prove?[/INS]The homeowner would generally need to prove more than just bad workmanship or a broken promise; they must show that the LLC’s owner dominated the company and used that control to commit a wrong that harmed the homeowner. Examples might include using the company to take deposits knowing it was insolvent, diverting project funds for personal use instead of paying subcontractors, or deliberately undercapitalizing the company so it could never meet its obligations. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Is an owner’s close control over a small LLC, by itself, enough for a court to pierce the corporate veil?[/INS]No. Many small businesses are closely held and naturally involve strong owner control, but veil piercing typically requires proof that this control was abused to commit a fraud, evade obligations, or work some other injustice. Without that wrongful use of the company, courts are reluctant to disregard the separate legal status of the LLC or corporation. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What kinds of facts do courts look at when deciding whether to pierce the corporate veil of a construction company?[/INS]Courts often look at whether corporate formalities were followed, whether the business was adequately capitalized, whether the owner commingled personal and company funds, and whether company assets were used for personal expenses. They also consider whether using the company form allowed the owner to perpetrate a fraud or injustice, such as leaving creditors unpaid while the owner benefited personally. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How can a small business owner help preserve the liability protection of their LLC or corporation?[/INS]A small business owner can help preserve the “veil” by treating the company as a separate entity: keeping separate bank accounts, documenting major decisions, signing contracts in the company’s name, and avoiding use of company funds for personal expenses. Adequate capitalization—having some realistic level of funds for operations—also helps show the company is not just a shell. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How does the doctrine of piercing the corporate veil protect consumers or subcontractors from abusive shell companies?[/INS]The doctrine allows courts, in unusual cases, to move past a nominal LLC or corporation and reach the personal assets of those who abused it to commit a wrong. This protects consumers and subcontractors from being left without a remedy when an owner deliberately uses a company as a shield while siphoning out value or engaging in deceptive practices. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.If a sole shareholder regularly pays personal credit card bills and vacations from the company bank account, what risk does that create regarding veil piercing?[/INS]Using company funds to pay personal expenses is a classic example of commingling, which is one factor courts consider when deciding whether to pierce the corporate veil. While one factor alone may not be decisive, a pattern of treating the company account as the owner’s personal wallet suggests abuse of the corporate form and increases the risk that a court will disregard the usual liability shield. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Why do courts describe veil piercing as an “equitable” and fact-specific doctrine rather than a rigid rule?[/INS]Veil piercing is “equitable” because courts use it to reach a fair result when strict adherence to the corporate form would work an injustice. Instead of applying a mechanical test, judges weigh many circumstances—like control, abuse, fairness, and the expectations of the parties—and then decide whether it would be unfair to let the owner hide behind the company. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Why might a court dismiss veil-piercing claims at an early stage if the evidence of abuse is thin?[/INS]Because piercing the corporate veil is an extraordinary remedy, courts often require concrete allegations of domination plus wrongful conduct, not just bare assertions that the owner controlled the company. If the record shows no significant commingling, undercapitalization, or misuse of the entity, a court may decide there is “no basis” to hold the owner personally liable and dismiss the veil-piercing claim. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.In a fixed-price construction contract, who usually bears the risk of unforeseen difficulties, and what is the major exception when the owner provides detailed plans and specifications?[/INS]Generally, when a contractor agrees to perform work for a fixed sum, the contractor bears the risk of unforeseen difficulties and cannot demand more money just because the job turns out harder than expected. The major exception arises when the owner provides detailed plans and specifications that the contractor is required to follow; in many U.S. jurisdictions, the owner is treated as having impliedly warranted that those plans are adequate, so the owner—not the contractor—bears the risk of defects in them. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What is the implied warranty that arises when a contractor must build strictly according to the owner’s plans and specifications?[/INS]When an owner furnishes detailed plans and specifications and requires the contractor to follow them, many courts recognize an implied warranty that the plans and specifications are reasonably adequate for their intended purpose. If the contractor builds in compliance and the design itself is defective, the contractor is typically not responsible for the resulting failures; instead, the owner may be in breach of this implied warranty. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.If a building’s foundation fails because of a design error in the owner’s drawings, even though the contractor followed the drawings exactly, who is generally responsible for the cost of fixing it?[/INS]Where the contractor was obligated to follow the owner’s plans and did so without deviation, many courts place responsibility for a design error on the owner rather than the contractor. Under the implied warranty of the adequacy of the plans, the owner has effectively promised that the design will work if properly built, so the contractor can often recover extra costs or damages caused solely by the faulty design. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Are general “site visit” and “check the plans” clauses usually enough to shift the risk of design defects back to the contractor?[/INS]Typical clauses requiring the contractor to examine the site, review the drawings, and inform itself about the work are usually not read as forcing the contractor to guarantee the owner’s design. Courts often hold that such general language does not override the more specific implied warranty that the owner’s detailed plans and specifications, if followed, will be adequate. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Does the implied warranty of the owner’s design apply only to the main structure, or can it also cover related work such as relocation of utility lines or sewers required by the plans?[/INS]The implied warranty usually applies to all parts of the work the contract requires to be performed according to the owner’s plans and specifications, not just the visible “main” structure. If, for example, the contract directs the contractor to relocate a sewer in a particular way and that specified design proves inadequate, courts often treat that sewer work as an integral part of the contract that is covered by the implied warranty. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What can a contractor do if it discovers that following the owner’s specified design would create an unsafe condition or seriously endanger the work?[/INS]A cautious contractor should promptly notify the owner in writing that the specified design appears unsafe or defective, explain the risk, and request that the owner either change the design or accept responsibility for the consequences. Many contracts and court decisions recognize that when a contractor reasonably believes the specified design creates danger, it may be justified in suspending work until the issue is resolved rather than proceeding “at its peril.” This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.If a contractor relocates a sewer exactly as shown on the plans and later discovers a hidden obstruction in a connected sewer that causes flooding, is the contractor automatically stuck with all the loss?[/INS]Not necessarily. When the contractor faithfully followed the owner’s specifications and the problem arises from hidden conditions or defects in the overall design—such as an undisclosed obstruction in a connecting sewer—courts often place responsibility on the owner under the implied warranty of the plans, especially when the contractor had no reasonable way to discover the issue beforehand. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Can an owner require a contractor to repair a dangerous, owner-designed sewer at its own expense while simultaneously denying any responsibility for the design?[/INS]In many jurisdictions, a contractor is not required to proceed with work that is unsafe and caused by defects in the owner’s design if the owner refuses to accept responsibility. If the owner insists on denying all responsibility yet demands that the contractor continue and pay for repairs arising from the owner’s own specifications, a court may treat the owner’s stance as a breach that justifies the contractor in halting work and seeking damages. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.When an owner wrongfully cancels a construction contract after refusing to correct or take responsibility for defective plans, what kinds of damages might the contractor recover?[/INS]If the owner unjustifiably terminates the contract, the contractor may be entitled to recover the value of work performed, costs incurred that have not been paid, and, in many cases, the profit it would reasonably have earned had it been allowed to complete the job. Courts calculate this by comparing what the contractor should have received under the contract to what it actually received and then adding provable lost profits. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How can a small contractor document its reliance on an owner’s plans and specifications to preserve rights if something later goes wrong?[/INS]A contractor can preserve its position by keeping copies of all drawings and specifications, noting any design assumptions that mattered to its bid, and avoiding undocumented field changes that alter the design. When issues arise, it should promptly send written notices stating that it is following the owner’s plans, describing the problem, and reserving its rights to claim extra time or money if the design proves defective. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What is a “differing site conditions” clause and why is it important in construction contracts?[/INS]A differing site conditions clause is a contract provision that allows a contractor to seek an adjustment in price or time when actual physical conditions at the site differ materially from what the contract documents indicated or from conditions normally expected. It helps avoid contractors building large contingencies into their bids by giving them a mechanism to be compensated if unexpected subsurface problems or unusual conditions arise. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What is the basic difference between a Type I and a Type II differing site conditions claim?[/INS]A Type I differing site condition involves a situation where the actual site conditions differ materially from the conditions that the contract affirmatively represented or indicated. A Type II differing site condition covers unknown physical conditions that are unusual in nature and differ materially from those ordinarily encountered in similar work, even if the contract said little about them. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.If a contractor agrees to install a sewer in a tunnel under a river and later encounters an unexpected toxic substance that causes major extra costs, does a prior change order automatically mean the contractor waived its differing site conditions rights?[/INS]Not automatically. Courts carefully read the change order alongside the original contract; if the modification simply approves a new method but does not clearly state that the contractor is waiving protection under the differing site conditions clause for unforeseen hazards like toxic substances, the contractor may still be able to claim relief. The key question is whether the change order shows a clear intent to shift or waive that particular risk. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Why might a change order that says the contractor “assumes responsibility” for a new construction method still not waive the differing site conditions clause?[/INS]Courts often distinguish between assuming responsibility for the means and methods of performing the work and assuming the risk of unknown, extraordinary physical conditions underground. If the change order only addresses method and does not clearly reference or negate the differing site conditions clause, many judges will conclude that the contractor did not give up the contractual protection for unforeseen subsurface hazards. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What elements typically must a contractor show to succeed on a Type II differing site conditions claim?[/INS]For a Type II claim, a contractor usually must show that it did not know about the condition before bidding, that the condition could not have been anticipated from a reasonable site inspection or general experience, and that the condition is different in kind or degree from what is normally encountered in similar projects. It also needs to show that this unusual condition caused additional costs or delays. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Why does it matter that a subsurface condition, like a toxic substance, is “completely outside and beyond the intent and contemplation of the parties”?[/INS]If a condition is truly outside what both parties contemplated when they signed the contract, courts are more likely to treat it as a differing site condition rather than a risk silently allocated to one side. That phrasing supports the argument that the contractor did not knowingly assume that particular risk and that the contract’s differing site conditions clause should protect the contractor from bearing all the extra costs. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.When a contractor unexpectedly encounters hazardous material on a project, what steps should it generally take to preserve its rights under a differing site conditions clause?[/INS]The contractor should stop or slow work in the affected area if necessary for safety, promptly notify the owner in writing of the condition, and request instructions while documenting the discovery with photos, logs, and test results. It should also follow any specific notice and claim procedures in the contract, since missing deadlines or continuing work without notice can sometimes weaken or even waive a differing site conditions claim. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Does every difference between expected and actual site conditions automatically entitle a contractor to more money under a differing site conditions clause?[/INS]No. The difference must be material and must cause actual increased cost or delay; minor variations from expectations are usually treated as part of the normal risk of construction. In addition, the contractor often must show that it reasonably interpreted the contract’s indications and that it relied on those indications when bidding. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How do differing site conditions clauses benefit both owners and contractors when bids are being prepared?[/INS]These clauses encourage contractors to bid based on the conditions reasonably indicated in the contract rather than padding their prices with large contingencies for every imaginable surprise. Owners benefit from lower, more competitive bids, while contractors gain a contractual pathway to seek adjustments if truly unexpected or unusual conditions arise. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What risk does broad, exculpatory language in a change order create if it is interpreted as shifting all subsurface risk to the contractor?[/INS]If a change order or other amendment is read as shifting all risk of subsurface conditions to the contractor, the contractor may lose the protection of the differing site conditions clause and be stuck with large unexpected costs. That can devastate profit margins or even push a contractor into financial distress if it must deal with hazards like contaminated soil or unstable ground without compensation. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.In a construction subcontract, what does it mean when the subcontractor agrees to accept the project site “as-is” with respect to subsurface conditions?[/INS]Accepting the site “as-is” often means the subcontractor agrees to bear the risk of whatever subsurface or concealed conditions exist, whether or not they were anticipated. Such clauses can significantly limit the subcontractor’s ability to claim extra money for differing site conditions later, because the subcontractor has contractually taken on that risk at the agreed price. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.If a subcontract says the subcontractor assumes all risk of subsurface conditions and later discovers much more contaminated soil than expected, what happens to its ability to seek additional payment?[/INS]Where the contract clearly states that the subcontractor assumes all risk of existing subsurface conditions and will not receive additional compensation if conditions differ from any estimates, courts often enforce that allocation. In that situation, the subcontractor may have little or no contractual basis to demand more money just because quantities of contaminated soil turned out higher than expected, unless it can prove fraud or some other exceptional circumstance. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How does a merger or integration clause in a construction contract affect claims based on pre-bid soil reports or conversations?[/INS]A merger or integration clause states that the written contract is the complete and final agreement, superseding all prior negotiations or representations. When such a clause is present, courts are less likely to consider arguments that the contractor relied on earlier pre-bid statements or soil reports that were not incorporated into the contract, because those prior materials are treated as legally “merged” into the written document or discarded. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What is the practical effect of a “no-reliance” clause where a subcontractor states it is not relying on the contractor’s estimates of contaminated soil?[/INS]A no-reliance clause lets the upstream contractor argue later that the subcontractor cannot claim it was misled by soil estimates in pre-bid documents, because the subcontractor expressly promised it was relying on its own investigation. Such language can be powerful in litigation, as it undercuts claims of reasonable reliance on pre-contract representations about quantities or conditions. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Why does it matter that a subcontract has multiple consistent clauses assigning subsurface risk to the subcontractor?[/INS]When risk-shifting language appears repeatedly—in definitions of “Work,” warranties about site investigation, “as-is” provisions, and clauses disclaiming reliance on estimates—courts are more likely to conclude that the parties clearly intended to allocate that risk to the subcontractor. A single ambiguous sentence can sometimes be argued away, but a pattern of consistent clauses makes it harder to claim the risk allocation was unclear or unintended. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Does the fact that a subcontractor had only a short time to investigate the site automatically invalidate a clause that puts all subsurface risk on the subcontractor?[/INS]Generally, no; courts often enforce clear risk-allocation clauses even when the contractor had limited time to investigate, especially in commercial settings with sophisticated parties. However, if the time constraints were extreme or combined with evidence of deliberate concealment or misrepresentation by the other party, some courts may be more open to arguments that enforcing the clause would be unfair or contrary to public policy. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What practical steps can a contractor take when asked to sign a subcontract that puts all subsurface risks on it?[/INS]A prudent contractor can seek to negotiate narrower language—for example, excluding truly unknown hazardous materials—or ask for additional compensation to cover the extra risk. It may also request access to more detailed site information, consult with legal counsel, or decide not to bid if the risk seems disproportionate to the potential reward. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Are general statements that the contractor has “investigated the site” usually enough, by themselves, to waive differing site conditions protections?[/INS]General investigation clauses can be important but are not always sufficient by themselves to wipe out specific differing site conditions protections, particularly if the contract also contains a standard differing site conditions clause. Courts often give greater weight to specific risk-allocation clauses over broad, boilerplate language, though modern decisions sometimes enforce strong investigation clauses as part of an overall scheme that shifts risk. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How do explicit assumption-of-risk clauses interact with the implied warranty that the owner’s plans and specifications are adequate?[/INS]Explicit assumption-of-risk clauses can limit or override the implied warranty by clearly stating that the contractor accepts certain risks, even if they arise from problems in the plans or incomplete information. In many jurisdictions, when sophisticated parties clearly allocate a particular risk to one side in the written contract, courts will enforce that allocation rather than rely on implied warranties that might otherwise favor the contractor. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.If an owner intentionally hides a known major hazard at the site but the contract has an integration and no-reliance clause, can the contractor still claim it was defrauded?[/INS]In many U.S. jurisdictions, integration and no-reliance clauses do not fully protect a party that intentionally commits fraud. A contractor may still be able to bring a fraud claim if it can prove that the owner knowingly concealed or misrepresented critical information, though the contract language may make that claim harder to win and the outcome can be very jurisdiction-dependent. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Why should contractors pay close attention to scope-of-work addenda that specifically mention contaminated soil quantities or similar conditions?[/INS]Scope-of-work addenda often contain detailed statements about who bears the risk of certain conditions, such as whether the price includes all contaminated soil removal regardless of quantity. Because these provisions can override more general clauses, missing a single sentence in an addendum can mean the difference between having a valid claim for extra costs and being stuck with a massive, unanticipated cleanup bill. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What does it mean if a subcontract says the subcontractor may not use the owner’s pre-bid estimates of contaminated soil as the basis for any later claim?[/INS]Such a clause usually means the subcontractor cannot argue that the difference between actual and estimated quantities entitles it to more money, because it promised not to rely on those estimates for claims. The subcontractor is treated as having priced the work based on its own investigation and accepted the risk that the owner’s estimates might be wrong. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How does agreeing to a lump-sum price interact with allocation of risk for differing site conditions?[/INS]A lump-sum price means the contractor agrees to perform the defined scope for a fixed amount, but it does not by itself answer who bears the risk of unforeseen conditions—that depends on the other contract clauses. If the contract also states that the contractor assumes all subsurface risk and will not receive adjustments for differing conditions, the lump-sum price reinforces that the contractor must absorb overruns; if there is a differing site conditions clause, the lump sum coexists with a mechanism for adjustments when conditions qualify. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Why can a subcontractor’s warranty that it relied on its own investigation weaken its later claims for extra compensation due to site conditions?[/INS]By warranting that it relied on its own investigation rather than on the other party’s estimates or opinions, the subcontractor undercuts a key element of many misrepresentation or differing site conditions arguments—reasonable reliance. When a dispute arises, the other side can point to this warranty and argue that the subcontractor knowingly took on the risk of inaccuracies in any pre-bid information. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What is the impact of contract language stating that there will be no additional compensation for changed site conditions?[/INS]Language that expressly states there will be no extra payment for changed or differing site conditions is a strong signal that the contractor has accepted that risk as part of its price. If clearly written and agreed to by commercial parties, courts often enforce such clauses, leaving the contractor to absorb even large cost increases from unexpected conditions unless some separate legal theory—such as fraud or impossibility—applies. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.When reviewing a heavily risk-shifting subcontract, what should a small contractor consider asking a lawyer to look at before signing?[/INS]A contractor may want a lawyer to review all clauses dealing with site investigation, differing site conditions, contaminated materials, integration and no-reliance, limitation of liability, and waiver of claims. Counsel can help explain which risks are being pushed onto the contractor, whether the language is unusually harsh compared to standard industry contracts, and what changes or pricing adjustments might be appropriate. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Why do owners and developers often try to shift subsurface risk to contractors in modern construction contracts?[/INS]Owners and developers want cost certainty and may lack detailed knowledge of subsurface conditions or the appetite to absorb large unknown risks. By shifting risk to contractors—who may be better positioned to investigate and price those risks—they aim to avoid surprises in their budgets, though this can make bids higher and disputes more likely if problems arise. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What insurance-related issues should a contractor think about before agreeing to take on all subsurface risks?[/INS]A contractor should consider whether its existing insurance covers the kinds of risks being assumed, such as pollution liability for contaminated soil or extra costs due to collapse or water inflow. If coverage is limited or excluded, the contractor may need to purchase specialized policies or reconsider whether the contract’s risk allocation is acceptable at the offered price. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How might courts respond if risk-shifting clauses in a contract are unclear or conflict with each other?[/INS]If clauses conflict or are ambiguous, courts often apply rules of interpretation that favor the party who did not draft the contract, especially when that party has less bargaining power. In that situation, a judge might interpret the ambiguity to preserve standard protections like differing site conditions clauses rather than assume the contractor knowingly gave them up. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Why is it helpful for contractors to document pre-bid questions and answers about site conditions?[/INS]Written pre-bid questions and answers create a record of what information was requested and provided, which can later help show what both sides understood and contemplated about the site. If a dispute arises over whether a condition was truly unforeseen or who assumed a particular risk, these communications may support the contractor’s interpretation of the contract and its reliance on the owner’s responses. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Under older-style owner–architect agreements that spoke of “general administration” and “supervision of the work,” what duties might a supervising architect have had on a construction project?[/INS]Under such agreements, courts sometimes viewed the architect as having an active role in overseeing construction to ensure compliance with plans and specifications, with authority to stop work that was not being performed safely or correctly. When architects had both design responsibilities and broad contractual powers to supervise and control aspects of the work, some courts found they owed a duty to exercise reasonable care to prevent obviously dangerous construction practices. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.If an architect with broad supervisory authority sees a contractor using an obviously unsafe shoring method and says nothing, could the architect face liability if the structure collapses and injures workers?[/INS]In some jurisdictions, yes. When an architect is contractually engaged to supervise construction and has the right to halt unsafe work, courts have sometimes held that failing to intervene in the face of an obvious, serious danger can be a breach of the architect’s duty of care toward workers foreseeably at risk. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What typical contract language places the primary responsibility for job-site safety on the contractor rather than on the architect?[/INS]Construction contracts often state that the contractor must “take all necessary precautions for the safety of employees,” comply with applicable safety laws, maintain safeguards, post warning signs, and designate a responsible person for accident prevention. These provisions make it clear that, as between contractor and owner or architect, the contractor bears primary responsibility for day-to-day site safety and methods of construction. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Why did professional organizations revise standard architect contracts to emphasize that architects are not responsible for construction means, methods, or safety programs?[/INS]Architects and their insurers were concerned that earlier language about “supervision” and “inspection” made it easier for courts to impose tort liability on architects for job-site accidents beyond their expertise or control. Revising the contracts to clarify that the contractor is solely responsible for means, methods, sequences, and safety programs was intended to align the architect’s legal duties with its design-focused role and reduce exposure to safety-related claims. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Under modern architect-owner agreements, what is the architect’s typical role during site visits?[/INS]Modern agreements often state that the architect will visit the site at reasonable intervals to observe the progress and general quality of the work and to determine if it appears to conform to the contract documents, but the architect is not required to make continuous, exhaustive inspections. The architect is expected to keep the owner informed and to “endeavor to guard” the owner against defects and deficiencies, while explicitly not being responsible for controlling construction means, methods, or safety programs. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Can an architect ever be liable for a worker’s injuries on a job site even if the contract disclaims responsibility for safety?[/INS]In many jurisdictions, contract disclaimers greatly reduce the architect’s exposure but do not provide absolute immunity. If an architect actually takes control of construction means and methods, gives specific directions that create a dangerous condition, or otherwise acts beyond the limited role described in the contract, a court may still find a duty and potential liability despite the disclaimer. This is general information about U.C. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What do modern architect contract clauses mean for an owner who wants very close, day-to-day supervision of construction?[/INS]These clauses signal that standard architect services do not include continuous on-site supervision or direct control of the contractor’s work. An owner who wants more intensive oversight may need to hire additional personnel, such as a clerk-of-the-works or owner’s representative, to be on site daily, and should adjust expectations and contracts accordingly rather than assuming the architect is filling that role. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What is a “clerk-of-the-works,” and how can that role change the level of oversight on a project?[/INS]A clerk-of-the-works is a person engaged, often through the architect, to be on site continuously to observe construction, check compliance with drawings, and report problems to the architect and owner. Hiring such a person can significantly increase day-to-day oversight compared with the architect’s normal periodic visits, but the contract should clearly define who employs the clerk, what authority they have, and how their observations relate to the contractor’s responsibilities. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Does the presence of an architect on a project relieve the contractor of its contractual duty to maintain a safe job site?[/INS]No. Even when an architect visits the site regularly or has broad authority under the contract, construction contracts almost always make the contractor primarily responsible for job-site safety and compliance with safety laws. The contractor cannot defend a safety failure simply by arguing that the architect also should have caught or prevented the unsafe condition. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.If a contract says the contractor must repair any damage “except such as may be directly due to errors in the contract documents or caused by the owner’s agents,” what does that mean for responsibility when a design error causes damage?[/INS]This language indicates that the contractor is not responsible for damage that stems directly from errors in the drawings or specifications prepared for the owner or from acts of the owner’s agents, such as architects or engineers. In those cases, the financial responsibility for correcting the damage is more likely to fall on the owner and potentially its design professionals, not on the contractor who simply followed the faulty documents. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How can safety statutes or special construction laws impose duties on owners or architects beyond what the contract itself says?[/INS]Some jurisdictions have statutes that place non-delegable safety duties on owners, contractors, or others involved in construction, meaning those duties exist regardless of how contracts allocate responsibility. For example, a statute aimed at protecting workers might allow injured employees to sue owners or supervising professionals even if the contract tried to shift all safety obligations to the contractor. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How can an injured worker sometimes sue an architect or owner even though workers’ compensation protects their direct employer from suit?[/INS]Workers’ compensation laws generally bar suits against the worker’s direct employer but do not automatically shield third parties like owners, architects, or other contractors. If those third parties owed an independent duty of care—by statute or by their own conduct—and breached it, the injured worker may bring a separate negligence or statutory claim against them, subject to the details of state law. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Why should architects be cautious about actually directing a contractor’s day-to-day construction methods on site?[/INS]When architects step beyond a limited observation role and begin directing means and methods—such as specifying how scaffolding must be erected or how shoring should be installed—they may create a duty to ensure those methods are reasonably safe. That can increase their exposure to liability for accidents, undermining contractual disclaimers that say they are not responsible for construction means and methods. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How should safety concerns ideally flow between architect, owner, and contractor on a construction project?[/INS]Typically, workers raise safety concerns with the contractor’s site supervision, who is responsible for corrective action under the contract and safety laws. If the architect or owner observes a dangerous condition, good practice is to immediately notify the contractor and, if necessary, exercise rights to stop work until the hazard is addressed, while documenting those communications in writing. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.For a small developer, how does the level of construction oversight relate to potential liability risk?[/INS]Minimal oversight reduces direct involvement in means and methods but can make it harder to catch problems early, while very hands-on oversight can increase the risk that an owner will be seen as controlling the work and thus sharing responsibility for accidents or defects. Developers often try to strike a balance by hiring qualified contractors, using clear contracts that allocate risk, and relying on design professionals for quality oversight without micromanaging day-to-day construction techniques. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.In the context of defective building materials like floor tiles, what is meant by “economic loss”?[/INS]Economic loss generally refers to purely financial harm, such as the cost of removing and replacing a defective product, lost profits, or diminished value of the building, without any personal injury or damage to other property. Many courts treat such losses as matters for contract and warranty law rather than tort law, especially in commercial construction settings. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.If wood floor tiles delaminate and fail throughout an apartment complex but only the tiles themselves are damaged, is the owner’s claim usually treated as a tort claim or a contract/warranty claim?[/INS]When the only harm is that the product fails to perform as expected and the owner must pay to repair or replace it, many courts treat the situation as a contract or warranty issue, not a tort. The owner’s remedy is typically governed by the agreements and warranties negotiated with contractors, subcontractors, and manufacturers, rather than by strict products liability. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Why do many courts limit strict products liability when a defective construction product only injures itself and causes economic loss?[/INS]Strict products liability was developed to protect people and their property from unsafe products, not to guarantee that every product meets its buyer’s economic expectations. When only the product itself fails, courts often reason that the parties could have negotiated warranties and risk allocations in their contracts, so they confine the dispute to contract law instead of expanding tort liability. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How can building owners protect themselves contractually against the risk that specified products, like flooring tiles, will be defective or wear out early?[/INS]Owners can negotiate express warranties on materials and workmanship, require that specific manufacturers provide direct warranties to the owner, and include contractual rights to repair or replacement if products fail within a defined period. They can also address allocation of risk among the owner, general contractor, subcontractors, and suppliers so that if a product is defective, there is a clear path to recover the cost of fixing it from the responsible party. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.When might strict products liability still apply in a construction materials case despite the economic loss rule?[/INS]Strict products liability may still apply when a defective product used in construction causes personal injuries or damages other property beyond the product itself, such as when faulty tiles cause residents to fall and suffer injuries or when a defective component causes a fire that damages the building. In those circumstances, courts are more willing to treat the case as a tort, because the harm goes beyond disappointed commercial expectations and implicates safety concerns. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What is indemnification between a subcontractor and a manufacturer in the context of defective construction products?[/INS]Indemnification is a legal mechanism that allows one party who paid damages—such as a subcontractor held liable to the owner—to shift those losses to another party, like the manufacturer that produced the defective product. The subcontractor may claim that the manufacturer breached an implied or express warranty and therefore should reimburse the subcontractor for amounts it had to pay because of the product’s defect. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.If a flooring subcontractor installs tiles made by a manufacturer and later must pay the owner for replacing them due to a defect, how might the subcontractor seek to recover its losses from the manufacturer?[/INS]The subcontractor could bring an indemnification or warranty claim against the manufacturer, arguing that the tiles were defective when supplied and that the manufacturer’s breach of implied or express warranty caused the subcontractor to incur damages. Whether this succeeds can depend on the contractual relationships, any disclaimers, and evidence about who controlled the product’s design and manufacturing process. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How can a subcontractor’s partial ownership interest in a manufacturer complicate an indemnity claim for defective products?[/INS]If a subcontractor has a significant ownership stake in the manufacturer, a court may view the subcontractor as more than a passive buyer and may scrutinize whether it shared responsibility for the product’s design or marketing. This overlap can make it harder for the subcontractor to portray itself as an innocent party entitled to shift all loss to the manufacturer, and may limit or defeat indemnity claims depending on the facts. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Why might a court allow a subcontractor’s implied warranty or indemnity claim against a manufacturer to go to a jury even if the owner’s tort claim is limited by the economic loss rule?[/INS]The owner’s tort claim focuses on whether strict liability is the right tool for recovering economic loss, while the subcontractor’s indemnity claim concerns how the costs should be allocated among parties in the chain of distribution under contract and warranty principles. Courts may decide that a jury should determine whether the manufacturer breached warranties owed to the subcontractor, even though the owner itself is largely limited to contract remedies and cannot sue the manufacturer in tort for pure economic loss. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How is an owner’s direct claim for product failure different from a subcontractor’s indemnity claim arising from the same defect?[/INS]The owner’s direct claim is about whether it can recover its repair or replacement costs and from whom, while the subcontractor’s indemnity claim is about reallocating those costs among the parties that supplied and installed the product. Even if the owner’s lawsuit is resolved under contract law, questions may remain about whether the subcontractor or manufacturer ultimately bears the economic burden, which can be sorted out through indemnity or contribution claims between those parties. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How does the economic loss rule encourage careful contract drafting in construction projects?[/INS]Because courts often limit tort remedies for pure economic loss, parties know that their primary protection lies in the contracts and warranties they negotiate ahead of time. This encourages owners, contractors, and suppliers to think carefully about risk allocation, warranty scope, and remedies for product failure when drafting agreements, rather than relying on broad tort theories to fill gaps after a problem arises. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What risks does a building owner face if it relies solely on tort theories like strict products liability instead of negotiating strong warranties for construction materials?[/INS]If a product fails but only causes economic loss—like the cost of replacing defective tiles—courts may limit the owner to contract and warranty remedies, denying tort recovery altogether. Without strong warranties, the owner may find that no party is clearly obligated to cover all of its losses, leaving the owner to absorb significant repair costs. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Why is documenting product approvals and substitutions important in a construction project?[/INS]When a specified product is substituted, written records of the approval process show who proposed the change, what conditions were attached, and what representations were made about performance. If the substitute later fails, these documents can be crucial in determining whether responsibility lies with the contractor who pushed for the change, the architect who approved it, the manufacturer, or some combination of them. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How might limitation-of-liability clauses in product sales contracts interact with obligations in construction contracts that use the product?[/INS]A manufacturer’s sales contract may limit remedies to repair or replacement or cap damages, while the construction contracts upstream might impose broader liability on contractors to the owner. This mismatch can leave contractors or subcontractors exposed to owner claims that exceed what they can recover from the manufacturer, making it important to align warranties and liability limits across the contract chain. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What business considerations should a small contractor weigh when deciding whether to use a manufacturer’s substitute product instead of the one originally specified?[/INS]The contractor should consider the reliability and track record of the substitute product, the scope of the manufacturer’s warranties, any changes in installation methods, and potential liability if the substitute fails. It should also assess whether it is taking on extra risk by promoting the change and whether the price difference justifies that risk, possibly seeking written agreements that clarify responsibility if problems arise. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What is the purpose of requiring a payment bond or guaranty when a private project is built on publicly owned land?[/INS]Payment security requirements are designed to protect contractors and subcontractors who cannot record mechanics’ liens on public property by ensuring there is an alternative source of funds if the project owner fails to pay. A bond or guaranty provides a backstop so that those furnishing labor and materials have a realistic way to recover what they are owed even if the project entity becomes insolvent. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How does a payment bond or enforceable guaranty help a contractor if the developer runs out of money?[/INS]If the developer cannot pay, the contractor can make a claim on the bond or enforce the guaranty against the party promising to stand behind the payment obligations, such as a parent company or surety. This can be critical where the contracting entity is a thinly capitalized special-purpose company with few assets of its own. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.If a developer on public land offers a written corporate guaranty instead of a surety bond, could that still satisfy payment protection requirements for contractors?[/INS]In some cases, courts may accept a properly structured guaranty as meeting the purpose of statutes that call for security to assure payment, especially if the guaranty clearly obligates a solvent entity to pay contractors if the project company defaults. The key question is whether the guaranty provides real, enforceable protection equivalent in substance to a bond, not just whether it uses the word “bond.” This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Why are veil-piercing claims often harder to win in disputes between sophisticated business entities than in consumer cases?[/INS]Courts expect sophisticated parties to protect themselves through careful contracting, due diligence, and negotiated security like bonds or guarantees. When entities with lawyers and bargaining power voluntarily deal with a thinly capitalized project company, judges are more reluctant to disregard corporate separateness unless there is clear evidence of fraud or abuse, preferring to enforce the risk allocation those parties chose. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Does the mere failure of a project or insolvency of an owner automatically justify piercing the corporate veil to reach affiliated companies?[/INS]No. Business failure alone is not enough to pierce the corporate veil; many legitimate ventures fail without any wrongdoing. Veil piercing typically requires proof of misuse of the corporate form—such as siphoning assets or using the company as a sham—not just the fact that creditors are left unpaid when the project goes badly. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What risks does a contractor face when signing a contract with a thinly capitalized special-purpose entity that owns no other assets and has no guaranty?[/INS]If the special-purpose entity runs out of money or goes bankrupt, the contractor may have little realistic chance of collecting unpaid balances or change orders, even if it wins in court. Without a bond, guaranty, or other security, the contractor’s judgment may be against an empty shell, leaving it to absorb significant losses. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What steps can a contractor take to protect itself against the risk that the owner or developer will be unable to pay?[/INS]Contractors can request payment bonds, corporate or personal guarantees, letters of credit, or other security from entities with real assets. They can also negotiate favorable payment schedules, limit how much work they perform before receiving payment, and conduct basic financial due diligence on the owner or developer before committing to the project. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Why is it important for a party seeking to pierce the corporate veil to plead specific instances of wrongdoing rather than just alleging “domination”?[/INS]Courts often view bare assertions of domination as insufficient because many owners legitimately control their companies. To get past early motions, the claimant usually must allege concrete facts showing misuse of that control—such as commingling funds, ignoring corporate formalities, or using the company to perpetrate a fraud—that connect the domination to the harm suffered. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.If a contractor’s complaint merely states that the parent company “dominated” the project entity without describing any specific abuses, is that likely to support veil piercing?[/INS]Standing alone, a generic allegation of domination is usually not enough; courts typically require more detailed facts that, if true, would show abuse of the corporate form. Without those specifics, judges may dismiss the veil-piercing claim, leaving the contractor limited to whatever security or remedies it negotiated at the contract stage. This is general information about U.Storage. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How does the existence of a valid guaranty reduce the practical need to pierce the corporate veil in a construction dispute?[/INS]If a solvent parent company has already guaranteed the project entity’s obligations, the contractor can typically sue directly on that guaranty to recover what it is owed. Because the guaranty provides a clear contractual route to reach the parent’s assets, there is less reason to invoke the more extreme and uncertain remedy of veil piercing. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.When reading a construction contract, how can a contractor figure out who bears the risk of subsurface conditions?[/INS]A contractor should look for clauses labeled “Differing Site Conditions,” “Site Investigation,” “As-Is,” “Assumption of Risk,” and similar headings, as well as warranties about having investigated the site and any language limiting claims for changed conditions. By reading these provisions together, it can usually tell whether the contract preserves traditional protection for unforeseen conditions or shifts most of that risk onto the contractor. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How should a contractor interpret statements that pre-bid soil data are provided “for information only” and are not guaranteed?[/INS]Such statements signal that the owner does not intend to warrant the accuracy of the data and that the contractor should not rely on the information as a basis for claims. While the contractor may still use the data to help prepare its bid, “for information only” language often supports the owner’s argument that the contractor accepted the risk of inaccuracies and must rely on its own investigation. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.If an owner hides a known major hazard at the site despite strong disclaimers and integration clauses, what kind of claim might still be available to the contractor?[/INS]Even with robust disclaimers, many courts allow a contractor to bring a fraud claim if it can prove the owner intentionally concealed or misrepresented a material hazard that the contractor could not reasonably discover. Success is far from guaranteed and depends heavily on state law and evidence, but fraud is one of the main ways contractors try to get around otherwise enforceable contractual risk-shifting clauses. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Why is prompt written notice so important when a contractor encounters a possible differing site condition?[/INS]Most construction contracts require timely written notice as a condition of recovering additional compensation or time for differing site conditions. Prompt notice gives the owner a fair chance to inspect the condition, decide how to respond, and mitigate costs; failing to give it can be treated as a waiver of the right to claim or may weaken the contractor’s case by suggesting the condition was not truly unexpected or serious. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.How can a contractor preserve its right to stop work when it believes the design has created an unsafe condition?[/INS]The contractor should immediately notify the owner (and architect, if applicable) in writing, explaining the unsafe condition and linking it to the design, and request clarification or changes before proceeding. Many contracts and safety laws support stopping work in the face of serious danger, and documenting the reasons and communications helps show that the contractor acted reasonably and did not abandon the job without cause. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What basic steps should a small construction business take to maintain the liability protection provided by forming an LLC or corporation?[/INS]The business should keep separate bank accounts, sign contracts in the company’s name, maintain some formal records and capitalization, and avoid treating company assets as personal funds. It should also use written contracts with clients that clearly identify the company—not the owner personally—as the contracting party, which helps show that everyone understood the deal was with the entity. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What key contract clauses should architects pay attention to that affect whether they might be blamed for job-site safety problems?[/INS]Architects should carefully review clauses describing their role during construction—especially sections on supervision, site visits, authority to stop work, and responsibility (or non-responsibility) for safety, means, and methods. Clear language stating that the contractor is solely responsible for safety and that the architect does not control construction methods can help limit unexpected liability, though it does not excuse truly negligent conduct. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What risk does a building owner face if repair costs for a defective product are not clearly covered by warranties in the construction contracts?[/INS]If warranties are weak, expired, or limited, the owner may find that no party is contractually obligated to pay for replacing or repairing a defective product that only caused economic loss. In that situation, tort claims may be barred by the economic loss rule, leaving the owner to shoulder much or all of the cost of correcting defects out of pocket. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.What is the basic idea of the implied warranty of merchantability for construction materials?[/INS]The implied warranty of merchantability, where it applies, is a background rule that goods sold by a merchant must be of fair average quality and fit for their ordinary, everyday purposes. For construction materials, that generally means the products should perform as commonly expected for that type of product when properly installed and used, unless the seller clearly and lawfully disclaims that warranty. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>",
  "<s>[INS]@ESE577.Before agreeing to a clause that gives it all risk of subsurface conditions and no extra pay for surprises, what business questions should a subcontractor ask itself?[/INS]A subcontractor should ask whether it has enough information and expertise to reasonably price that risk, whether the contract price adequately compensates for worst-case scenarios, and whether its insurance and financial reserves could handle a major surprise. It should also consider whether it can negotiate better terms, whether the overall relationship with the upstream contractor or owner justifies accepting the risk, and what impact a serious unforeseen problem could have on the company’s survival. This is general information about U.S. law, not legal advice. Laws vary by state and situation; talk to a licensed attorney about your specific issue.</s>"
]

In [1]:
"""
# This block contains the code to interact with the Google Gemini 2.5 Flash API
# to request questions and answers for your data. It loops through a list of
# paragraphs and requests Gemini to create one question for each paragraph indi-
# vidually. Your main job here is to write appropriate prompts that lead Gemini
# to generate useful questions. You can also consider generating more/fewer
# questions per paragraph, or merging paragraphs if you think that will help.
# Be sure to document any changes you make in your report!

# TODO: create your own Gemini API key, and either paste it here (and then
# remove it before turning in your report) or save it in a file and load it here.
geminiApiKey= "INSERT KEY HERE"
genai.configure(api_key=geminiApiKey)
cfg = genai.types.GenerationConfig(max_output_tokens=4000)
sys_msg_train = (
'''

 For this json line, strictly and ONLY reformat the line as so where you would replace <question> with what is inside of the quotes after {"input": in the json line, and replace <answer> with what is inside the quotes after "output": to convert the line as so:
 <s>[INS]@ESE577.<question>[/INS]<answer></s>

'''
)
print(sys_msg_train)
print()
model_train = genai.GenerativeModel('gemini-2.5-flash', system_instruction=sys_msg_train)
qa_pairs_train = []
for par in paragraphs[30:130]:
  qa_pairs_train.append(model_train.generate_content(par, generation_config=cfg).text)
  print(qa_pairs_train[-1])
"""

'\n# This block contains the code to interact with the Google Gemini 2.5 Flash API\n# to request questions and answers for your data. It loops through a list of\n# paragraphs and requests Gemini to create one question for each paragraph indi-\n# vidually. Your main job here is to write appropriate prompts that lead Gemini\n# to generate useful questions. You can also consider generating more/fewer\n# questions per paragraph, or merging paragraphs if you think that will help.\n# Be sure to document any changes you make in your report!\n\n# TODO: create your own Gemini API key, and either paste it here (and then\n# remove it before turning in your report) or save it in a file and load it here.\ngeminiApiKey= "INSERT KEY HERE"\ngenai.configure(api_key=geminiApiKey)\ncfg = genai.types.GenerationConfig(max_output_tokens=4000)\nsys_msg_train = (\n\'\'\'\n\n For this json line, strictly and ONLY reformat the line as so where you would replace <question> with what is inside of the quotes after

In [27]:
# Create HuggingFace datasets
dataset_train = Dataset.from_pandas(pd.DataFrame(qa_pairs_train, columns=["text"]))
dataset_val = Dataset.from_pandas(pd.DataFrame(qa_pairs_val, columns=["text"]))
dataset = DatasetDict({"train": dataset_train, "test": dataset_val})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text'],
        num_rows: 30
    })
})


In [ ]:
# Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
# save login credentials for conveneince
!git config --global credential.helper store
!hf auth login --token <INSERT TOKEN HERE> --add-to-git-credential

Token is valid (permission: write).
The token `ese577Project` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `ese577Project`


In [34]:
# Load the model -- Skeleton
# previously was mistralai/Mistral-7B-Instruct-v0.2
base_model = "mistralai/Mistral-7B-Instruct-v0.3"
bnb_config = BitsAndBytesConfig( # given in official docs for 4 bit quantization
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained( # given in official docs for model setup
    base_model, quantization_config=bnb_config, device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [35]:
# Tokenize the data
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.bos_token, tokenizer.eos_token

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

('<s>', '</s>')

In [ ]:
# LoRA config -- Skeleton
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    '''
      TODO: find an appropriate configuration to use here for LoRA
    '''
)
model = get_peft_model(model, peft_config)

In [ ]:
# Hyperparameters -- Skeleton
training_arguments = TrainingArguments(
    '''
      TODO: find an appropriate configuration to use here for training
    '''
)

In [ ]:
# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length=None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False
)
trainer.train()

In [ ]:
# Save the model
trainer.model.save_pretrained("ESE577_chatbot")
model.config.use_cache = True
model.eval()

In [ ]:
# Run the model locally
logging.set_verbosity(logging.CRITICAL)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, truncation=True)
def build_prompt(question):
  prompt = f"<s>[INS]@ESE577.{question}.[/INS]"
  return prompt

while True:
  question = input("Enter your ESE577-related question (hit Enter to exit): ").strip()
  if not question:
    break
  prompt = build_prompt(question)
  answer = pipe(prompt)
  print(answer[0]["generated_text"])
  print()
